## Setup

In [ ]:
import requests
from os import getenv
import base64
from dotenv import load_dotenv

In [ ]:
load_dotenv()
IDENTITY_DOMAINS_HOST = getenv('IDENTITY_DOMAINS_HOST')
CLIENT_ID = getenv('CLIENT_ID')
CLIENT_SECRET = getenv('CLIENT_SECRET')
USERNAME = getenv('USERNAME')
PASSWORD = getenv('PASSWORD')
AGENT_ID = getenv('GEN_AI_AGENTS_ID')
API_ENDPOINT = getenv('GEN_AI_AGENTS_ENDPOINT')

In [ ]:
#  Obtain an access token using resource owner password credentials grant
well_known_response = requests.get(
    url=IDENTITY_DOMAINS_HOST + '/.well-known/openid-configuration'
)

if not well_known_response.status_code == 200:
    raise Exception('Failed to obtain openid connect configuration')

token_endpoint = well_known_response.json().get('token_endpoint')
headers = {
    'Authorization': 'Basic ' + base64.b64encode(f'{CLIENT_ID}:{CLIENT_SECRET}'.encode()).decode(),
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}
body = f'grant_type=client_credentials&scope={API_ENDPOINT}/genaiagent'

token_endpoint_response = requests.post(
    url=token_endpoint,
    headers=headers,
    data=body,
    verify=True
)

if not token_endpoint_response.status_code == 200:
    raise Exception('Failed to obtain access token')

access_token = token_endpoint_response.json().get('access_token')


In [ ]:
# RAG - OCI Search Service with OpenSearch
chat_request_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-type': 'application/json'
}
chat_request_details = {
    'agentId': AGENT_ID,
    'query': 'What is region in OCI?'
}

response = requests.post(
    url=f'{API_ENDPOINT}/20240331/actions/chat',
    json=chat_request_details,
    headers=chat_request_headers,
    verify=False
)

print(response.json())

In [ ]:
chat_request_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-type': 'application/json'
}
chat_request_details = {
    'agentId': AGENT_ID,
    'query': 'Hi, how are you?'
}

response = requests.post(
    url=f'{API_ENDPOINT}/20240331/actions/chat',
    json=chat_request_details,
    headers=chat_request_headers,
    verify=False,
)

print(response.json())

In [ ]:
chat_request_details['shouldReturnSearchDocuments'] = True

response = requests.post(
    url=f'{API_ENDPOINT}/20240331/actions/chat',
    json=chat_request_details,
    headers=chat_request_headers,
    verify=False,
)

print(response.json())